<a href="https://colab.research.google.com/github/yongghongg/stock-screener/blob/main/automate_stock_screener_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A demo Colab Notebook for my article: 
https://levelup.gitconnected.com/automate-your-stock-screening-using-python-9107dda724c3

In [ ]:
# install required libraries (on colab)
!pip install yfinance
!pip install bs4
!pip install requests
# import required libraries 
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import email
import pandas as pd

In [ ]:
def get_stock_list():
  # this is the website we're going to scrape from
  url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
  response = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find(id = "MainContent2_tbAllStock")
  # return the result (only ticker code) in a list
  stock_list = table.find_all('a')
  return [stock_code.get('href')[-4:] for stock_code in stock_list]

def get_stock_price(code):
  # you can change the start date
  data = yf.download(code, start="2021-01-01", threads= False)
  return data

def add_EMA(price, day):
  return price.ewm(span=day).mean()

def add_STOCH(close, low, high, period, k, d=0): 
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
      return STOCH_K
    else:
      STOCH_D = STOCH_K.rolling(window=d).mean()
      return STOCH_D

def check_bounce_EMA(df):
  candle1 = df.iloc[-1]
  candle2 = df.iloc[-2]
  cond1 = candle1['EMA18'] > candle1['EMA50'] > candle1['EMA100']
  cond2 = candle1['STOCH_%K(5,3,3)'] <= 30 or candle1['STOCH_%D(5,3,3)'] <= 30
  cond3 = candle2['Low'] < candle2['EMA50'] and \
          candle2['Close'] > candle2['EMA50'] and \
          candle1['Low'] > candle1 ['EMA50']
  return cond1 and cond2 and cond3

# a list to store the screened results
screened_list = [] 
# get the full stock list
stock_list = get_stock_list()
for stock_code in stock_list:

  print(stock_code) # remove this if you dont want the ticker to be printed
  try: 
    # Step 1: get stock price for each stock
    price_chart_df = get_stock_price(stock_code + ".KL") # edit/remove ".KL" for other exchange market

    # Step 2: add technical indicators (in this case EMA)
    close = price_chart_df['Close']
    low = price_chart_df['Low']
    open = price_chart_df['Open']
    high = price_chart_df['High']
    price_chart_df['EMA18'] = add_EMA(close,18)
    price_chart_df['EMA50'] = add_EMA(close,50)
    price_chart_df['EMA100'] = add_EMA(close,100)
    price_chart_df['STOCH_%K(5,3,3)'] = add_STOCH(close, low, high, 5, 3)
    price_chart_df['STOCH_%D(5,3,3)'] = add_STOCH(close, low, high, 5, 3, 3)

    # if all 3 conditions are met, add stock into screened list
    if check_bounce_EMA(price_chart_df):
      screened_list.append(stock_code)
      print(screened_list)
  except Exception as e:
    print(e)

## this cell may take a few minutes to finish

In [ ]:
# configure email and message
msg = email.message_from_string(", ".join(screened_list))
msg['From'] = 'YOUR_EMAIL@gmail.com' # change to your email 
msg['To'] = 'YOUR_EMAIL@gmail.com'
msg['Subject'] = "EMA Bounce Result for Today!"

s = smtplib.SMTP("smtp.gmail.com",587)
## for yahoo mail user: s = smtplib.SMTP("smtp.mail.yahoo.com",587) 
## for hotmail user: s = smtplib.SMTP("smtp.live.com",587)
s.ehlo() 
s.starttls()
s.ehlo()
s.login(email_from,"YOUR_PASSWORD") # change to your password
s.sendmail(email_from, [email_to] + [email_cc], msg.as_string())
s.quit()

In [ ]:
# for US stock list
# credit to https://github.com/shilewenuw/get_all_tickers/issues/12
def get_US_stock_list(exchange):
  headers = {
      'authority': 'api.nasdaq.com',
      'accept': 'application/json, text/plain, */*',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
      'origin': 'https://www.nasdaq.com',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.nasdaq.com/',
      'accept-language': 'en-US,en;q=0.9',
  }

  params = (
      ('tableonly', 'true'),
      ('limit', '25'),
      ('offset', '0'),
      ('download', 'true'),
      ('exchange', exchange)
  )

  r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
  data = r.json()['data']
  df = pd.DataFrame(data['rows'], columns=data['headers'])
  df_filtered = df[~df['symbol'].str.contains("\.|\^|\s")]
  return df_filtered['symbol'].tolist()

stock_list = get_US_stock_list('nyse') # or 'nasdaq' or 'amex'